### Assisgment:
Create a simple assistant that uses any LLM and should be pydantic, when we ask about any product it should give you two information product Name, product details tentative price in USD (integer). use chat Prompt Template.


In [1]:
# Import required libraries
import os
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

In [2]:
# Pydantic model for product information
class ProductInfo(BaseModel):
    """Model for product information response"""
    product_name: str = Field(description="The name of the product")
    product_details: str = Field(description="Detailed description of the product including features, specifications, etc.")
    tentative_price_usd: int = Field(description="Tentative price in USD as an integer")



ProductInfo.model_json_schema()

{'description': 'Model for product information response',
 'properties': {'product_name': {'description': 'The name of the product',
   'title': 'Product Name',
   'type': 'string'},
  'product_details': {'description': 'Detailed description of the product including features, specifications, etc.',
   'title': 'Product Details',
   'type': 'string'},
  'tentative_price_usd': {'description': 'Tentative price in USD as an integer',
   'title': 'Tentative Price Usd',
   'type': 'integer'}},
 'required': ['product_name', 'product_details', 'tentative_price_usd'],
 'title': 'ProductInfo',
 'type': 'object'}

In [3]:
if "GROQ_API_KEY" not in os.environ:
    api_key = input("Please enter your Groq API key: ")
    os.environ["GROQ_API_KEY"] = api_key

print("Groq API key configured!")

Groq API key configured!


In [4]:
# Initialize the Groq LLM
llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0.1,  # Low temperature for more consistent responses
    max_tokens=500
)

llm.model_name

'llama3-8b-8192'

In [5]:
from langchain.output_parsers import PydanticOutputParser

In [6]:
output_parser = PydanticOutputParser(pydantic_object=ProductInfo)

In [7]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", """
    You are a helpful product information assistant. When a user asks about a product, 
    you should provide accurate information according to the format instructions below.
    
    Important guidelines:
    - Provide realistic and accurate product information
    - Price should be a reasonable estimate in USD as an integer (no decimals)
    - Include detailed product specifications and features
    - If you're not sure about a specific product, provide general information about that type of product
    
    {format_instructions}
    """),
    ("human", "Tell me about: {product_query}")
])

In [8]:
prompt_template

ChatPromptTemplate(input_variables=['format_instructions', 'product_query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions'], input_types={}, partial_variables={}, template="\n    You are a helpful product information assistant. When a user asks about a product, \n    you should provide accurate information according to the format instructions below.\n    \n    Important guidelines:\n    - Provide realistic and accurate product information\n    - Price should be a reasonable estimate in USD as an integer (no decimals)\n    - Include detailed product specifications and features\n    - If you're not sure about a specific product, provide general information about that type of product\n    \n    {format_instructions}\n    "), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['product_query'], input_types={}, partial_variables={}, template='Tell me about: {produc

In [9]:
output_parser.get_format_instructions()


'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"description": "Model for product information response", "properties": {"product_name": {"description": "The name of the product", "title": "Product Name", "type": "string"}, "product_details": {"description": "Detailed description of the product including features, specifications, etc.", "title": "Product Details", "type": "string"}, "tentative_price_usd": {"description": "Tentative price in USD as an integer", "title": "Tentative Price Usd", "type": "integer"}}, "required": ["product_name", "product_details", "tentative_price_usd"]}

In [10]:
chain = (
    {
        "product_query": RunnablePassthrough(),
        "format_instructions": lambda _: output_parser.get_format_instructions()
    }
    | prompt_template
    | llm
    | output_parser
)

chain

{
  product_query: RunnablePassthrough(),
  format_instructions: RunnableLambda(...)
}
| ChatPromptTemplate(input_variables=['format_instructions', 'product_query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions'], input_types={}, partial_variables={}, template="\n    You are a helpful product information assistant. When a user asks about a product, \n    you should provide accurate information according to the format instructions below.\n    \n    Important guidelines:\n    - Provide realistic and accurate product information\n    - Price should be a reasonable estimate in USD as an integer (no decimals)\n    - Include detailed product specifications and features\n    - If you're not sure about a specific product, provide general information about that type of product\n    \n    {format_instructions}\n    "), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=

In [12]:
def get_product_info(product_query: str, show_raw_response: bool = False) -> ProductInfo:
    try:
        product_info = chain.invoke(product_query)
        
        # if show_raw_response:
        #     print(f"Raw LLM Response (parsed):\n{product_info}\n")
        
        return product_info
        
    except Exception as e:
        print(f"Error getting product information: {e}")
        return None


In [13]:
test_query = "iPhone 14 Pro"

In [14]:
result = get_product_info(test_query)

In [16]:
result.product_name, result.product_details, result.tentative_price_usd

('iPhone 14 Pro',
 "The iPhone 14 Pro is a flagship smartphone developed by Apple Inc. It features a 6.1-inch Super Retina XDR OLED display with a resolution of 1284 x 2778 pixels. The device is powered by Apple's A16 Bionic chip, which provides fast performance and efficient battery life. The iPhone 14 Pro has a quad-camera setup, consisting of a 48MP primary sensor, a 12MP front camera, and a telephoto lens with 2x optical zoom. It also features a 120Hz refresh rate, IP68 water resistance, and a stainless steel frame. The device runs on iOS 16 and supports 5G connectivity. It is available in four colors: Space Gray, Gold, Silver, and Sierra Blue.",
 999)

In [17]:
# Interactive function to query any product
def ask_about_product():
    """
    Interactive function to ask about any product
    """
    print("\n Product Information Assistant")
    print("Ask me about any product and I'll provide details!")
    print("Type 'quit' to exit\n")
    
    try:
        while True:
            try:
                user_input = input("What product would you like to know about? ")
                
                if user_input.lower() in ['quit', 'exit', 'q']:
                    print("Thanks for using the Product Assistant! 👋")
                    break
                    
                if user_input.strip():
                    print(f"\n🔍 Searching for information about: {user_input}")
                    print("-" * 60)
                    
                    try:
                        result = get_product_info(user_input)
                        if result:
                            print("\n Product Information:")
                            print(f"• Name: {result.product_name}")
                            print(f"• Details: {result.product_details}")
                            print(f"• Estimated Price: ${result.tentative_price_usd} USD")
                        else:
                            print(" Sorry, couldn't retrieve product information.")
                    except Exception as e:
                        print(f" Error: {e}")
                    
                    print("\n" + "="*60)
                else:
                    print("Please enter a product name or 'quit' to exit.")
                    
            except EOFError:
                # Handle Ctrl+D (EOF)
                print("\n\nExiting Product Assistant...")
                break
                
    except KeyboardInterrupt:
        # Handle Ctrl+C
        print("\n\nProduct Assistant interrupted. Goodbye!")
    except Exception as e:
        print(f"\n\nUnexpected error: {e}")
        print("Product Assistant stopped.")

print("Interactive function ready! Call ask_about_product() to start querying.")

Interactive function ready! Call ask_about_product() to start querying.


In [18]:
# To start the interactive assistant, uncomment and run: ask_about_product() and type `exit` to exit!
ask_about_product()


 Product Information Assistant
Ask me about any product and I'll provide details!
Type 'quit' to exit


🔍 Searching for information about: iphone 12
------------------------------------------------------------

 Product Information:
• Name: iPhone 12
• Details: The iPhone 12 is a flagship smartphone developed by Apple Inc. It features a 6.1-inch Super Retina HD display with a resolution of 1080 x 2536 pixels. The device is powered by Apple's A14 Bionic chip, which provides fast performance and efficient battery life. The iPhone 12 has a quad-camera setup with a 12-megapixel main camera, a 12-megapixel ultra-wide-angle camera, a 12-megapixel telephoto camera, and a time-of-flight camera. It also features a 7-megapixel front camera with a TrueDepth sensor. The device has a stainless steel frame and is available in six colors: Space Gray, Gold, Silver, Red, Blue, and Coral. It also features IP68 water and dust resistance, wireless charging, and fast charging capabilities.
• Estimated Pri

In [19]:
# Simple test of multiple products
test_products = ["MacBook Pro M3", "Tesla Model Y", "PlayStation 5"]

print("Testing multiple products:\n")
for product in test_products:
    print(f"🔍 Querying: {product}")
    result = get_product_info(product)
    if result:
        print(f" {result.product_name}: ${result.tentative_price_usd} : {result.product_details}")
    else:
        print(f" Failed to get info for {product}")
    print("-" * 40)

Testing multiple products:

🔍 Querying: MacBook Pro M3
 MacBook Pro M3: $1299 : The MacBook Pro M3 is a high-performance laptop designed for professionals and creatives. It features a 13.3-inch Retina display with a resolution of 2560 x 1600 pixels, powered by Apple's M3 chip. It has 8GB of RAM and 256GB of SSD storage. The laptop also features a Touch Bar with Touch ID, a FaceTime HD camera, and two Thunderbolt 3 (USB-C) ports. It runs on macOS Monterey and has a battery life of up to 10 hours.
----------------------------------------
🔍 Querying: Tesla Model Y
 Tesla Model Y: $39000 : The Tesla Model Y is a compact luxury crossover SUV produced by Tesla, Inc. It was unveiled in March 2019 and is based on the same platform as the Model 3 sedan. The Model Y is designed to be a more practical and spacious alternative to the Model 3, with a larger cargo area and seating for up to seven passengers. It features a range of up to 315 miles on a single charge, depending on the trim level and o

## How to Use the Product Assistant

### Option 1: Direct Function Call
```python
# Get information about a specific product
result = get_product_info("MacBook Pro M3")
if result:
    print(f"Product: {result.product_name}")
    print(f"Price: ${result.product_details}")
    print(f"Price: ${result.tentative_price_usd}")
```

### Option 2: Interactive Mode
```python
# Start interactive session
ask_about_product()
```

### Option 3: Test Multiple Products
```python
products = ["Tesla Model 3", "PlayStation 5", "Nike Air Max", "Samsung Galaxy S24"]
for product in products:
    info = get_product_info(product)
    if info:
        print(f"{info.product_name}: ${info.tentative_price_usd} : {result.product_details}")
```